In [1]:
! pip install -U tensorboardcolab

Requirement already up-to-date: tensorboardcolab in /usr/local/lib/python3.6/dist-packages (0.0.19)


In [2]:
from tensorboardcolab import *

tensorboard_colab = TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://2ab4d284.ngrok.io


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install xlrd

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xlrd

'''
Step 0: Suppress warning logs and 
define a variable to print out for a certain step
'''
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

controlling_step = 99

''' 
Step 1: Read in data from the .xls file
'''
DATA_FILE = 'drive/My Drive/Materials for training/Practical AI Bootcamp/Week 3/Data/fire_theft.xls'

book = xlrd.open_workbook(DATA_FILE, encoding_override='utf-8')
sheet = book.sheet_by_index(0)

number_of_rows = len(list(sheet.get_rows()))
data = np.asarray([sheet.row_values(i) for i in range(1, number_of_rows)])

'''
Step 2: Create placeholders for input X (number of fire) and label Y (number of theft)
'''
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

'''
Step 3: Create theta0 and theta1, initialized them to 0
'''
theta0 = tf.Variable(0.0, name='theta0')
theta1 = tf.Variable(0.0, name='theta1')

'''
Step 4: Define a hypothesis function to predict Y
'''
hypothesis_function = theta0 + theta1 * X

'''
Step 5: Use the square error as the loss function
'''
loss_function = tf.multiply(tf.divide(1,2), tf.reduce_mean(tf.pow(Y - hypothesis_function, 2)))
tf.summary.scalar('total_cost', loss_function)

'''
Step 6: Using gradient descent with learning rate of 0.001 to minimize loss
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss_function)

with tf.Session() as session:
    '''
    Step 7: Initialize the necessary variables, i.e. theta0 and theta1
    '''
    session.run(tf.global_variables_initializer())

    colab_writer = tensorboard_colab.get_writer();
    colab_writer.add_graph(session.graph)
    colab_writer.flush();
    
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./graphs/linear_regression', session.graph)

    '''
    Step 8: Train the model for 30,000 epochs
    '''
    for i in range(30000):
        session.run(optimizer, feed_dict={X: data.T[0], Y: data.T[1]})

        summary, cost = session.run([merged, loss_function], feed_dict={X: data.T[0], Y: data.T[1]})
        writer.add_summary(summary, i)
        
        colab_writer.add_summary(summary, i)
        colab_writer.flush();

        if i % controlling_step == 0:
            print("Epoch: {0}, cost = {1}, theta0 = {2}, theta1 = {3}".format(i + 1, cost,
                                                                          session.run(theta0), session.run(theta1)))

    '''
    Step 9: Prints the training cost, theta0, and theta1
    '''
    print("Optimization Finished!")
    training_cost = session.run(loss_function, feed_dict={X: data.T[0], Y: data.T[1]})
    print("Cost =", training_cost, "theta0 = ", session.run(theta0), "theta1 = ", session.run(theta1), '\n')

    '''
    Step 10: Plot the results
    '''
    # Graphic display
    plt.plot(data.T[0], data.T[1], 'ro', label='Original data')
    plt.plot(data.T[0], session.run(theta0) + session.run(theta1) * data.T[0], 'b', label='Fitted line')
    plt.xlabel('fires per 1000 housing units')
    plt.ylabel('thefts per 1000 population')
    plt.legend()
    plt.show()

# Close the writer when you finished using it
writer.close()
tensorboard_colab.close()


Epoch: 1, cost = 563.3204956054688, theta0 = 0.03366667032241821, theta1 = 0.5471643209457397
Epoch: 100, cost = 228.37046813964844, theta0 = 0.7072527408599854, theta1 = 2.1338961124420166
Epoch: 199, cost = 225.0644989013672, theta0 = 1.2786383628845215, theta1 = 2.1051149368286133
Epoch: 298, cost = 221.98654174804688, theta0 = 1.829979658126831, theta1 = 2.077343225479126
Epoch: 397, cost = 219.12063598632812, theta0 = 2.36197829246521, theta1 = 2.0505456924438477
Epoch: 496, cost = 216.45233154296875, theta0 = 2.875314950942993, theta1 = 2.024688482284546
Epoch: 595, cost = 213.9679412841797, theta0 = 3.3706438541412354, theta1 = 1.999738097190857
Epoch: 694, cost = 211.65478515625, theta0 = 3.8485960960388184, theta1 = 1.975663185119629
Epoch: 793, cost = 209.50106811523438, theta0 = 4.309781074523926, theta1 = 1.9524327516555786
Epoch: 892, cost = 207.49583435058594, theta0 = 4.754787921905518, theta1 = 1.9300172328948975
Epoch: 991, cost = 205.6288299560547, theta0 = 5.18418312